In [2]:
import ee
import geemap

In [3]:
ee.Authenticate()
ee.Initialize()

In [5]:
Mohe = ee.FeatureCollection('projects/earthengine-legacy/assets/projects/sat-io/open-datasets/geoboundaries/HPSCU-ADM3').filterMetadata(name="shapeName", operator="equals", value="Mohe County")
Emuer = ee.FeatureCollection("projects/sat-io/open-datasets/HydroAtlas/BasinAtlas/BasinATLAS_v10_lev06").filterBounds(Mohe.geometry().centroid())

In [6]:
print("How many areas in Mohe? ", Mohe.geometry().area().divide(1000000).format("%.2f").getInfo(), "Km^2")
print("How many areas in Emuer? ", Emuer.geometry().area().divide(1000000).format("%.2f").getInfo(), "Km^2")
print(
    "The proportion of Emuer to the Mohe: ", 
    Emuer.geometry().area().divide(1000000)
    .divide(Mohe.geometry().area().divide(1000000))
    .multiply(100).format("%.2f").getInfo(), "%"
)

How many areas in Mohe?  18409.04 Km^2
How many areas in Emuer?  16105.91 Km^2
The proportion of Emuer to the Mohe:  87.49 %


In [7]:
# https://developers.google.com/earth-engine/tutorials/community/sentinel-2-s2cloudless
def get_s2_sr_cld_col(aoi, start_month, end_month):
    # Import and filter S2 SR.
    s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
        .filterBounds(aoi)
        .filter(ee.Filter.calendarRange(start_month,end_month,'month'))
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER)))

    # Import and filter s2cloudless.
    s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
        .filterBounds(aoi)
        .filter(ee.Filter.calendarRange(start_month,end_month,'month')))

    # Join the filtered s2cloudless collection to the SR collection by the 'system:index' property.
    return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': s2_sr_col,
        'secondary': s2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))

def add_cloud_bands(img):
    # Get s2cloudless image, subset the probability band.
    cld_prb = ee.Image(img.get('s2cloudless')).select('probability')

    # Condition s2cloudless by the probability threshold value.
    is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds')

    # Add the cloud probability layer and cloud mask as image bands.
    return img.addBands(ee.Image([cld_prb, is_cloud]))

def add_shadow_bands(img):
    # Identify water pixels from the SCL band.
    not_water = img.select('SCL').neq(6)

    # Identify dark NIR pixels that are not water (potential cloud shadow pixels).
    SR_BAND_SCALE = 1e4
    dark_pixels = img.select('B8').lt(NIR_DRK_THRESH*SR_BAND_SCALE).multiply(not_water).rename('dark_pixels')

    # Determine the direction to project cloud shadow from clouds (assumes UTM projection).
    shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')));

    # Project shadows from clouds for the distance specified by the CLD_PRJ_DIST input.
    cld_proj = (img.select('clouds').directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST*10)
        .reproject(**{'crs': img.select(0).projection(), 'scale': 100})
        .select('distance')
        .mask()
        .rename('cloud_transform'))

    # Identify the intersection of dark pixels with cloud shadow projection.
    shadows = cld_proj.multiply(dark_pixels).rename('shadows')

    # Add dark pixels, cloud projection, and identified shadows as image bands.
    return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))

def add_cld_shdw_mask(img):
    # Add cloud component bands.
    img_cloud = add_cloud_bands(img)

    # Add cloud shadow component bands.
    img_cloud_shadow = add_shadow_bands(img_cloud)

    # Combine cloud and shadow mask, set cloud and shadow as value 1, else 0.
    is_cld_shdw = img_cloud_shadow.select('clouds').add(img_cloud_shadow.select('shadows')).gt(0)

    # Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input.
    # 20 m scale is for speed, and assumes clouds don't require 10 m precision.
    is_cld_shdw = (is_cld_shdw.focalMin(2).focalMax(BUFFER*2/20)
        .reproject(**{'crs': img.select([0]).projection(), 'scale': 20})
        .rename('cloudmask'))

    # Add the final cloud-shadow mask to the image.
    return img_cloud_shadow.addBands(is_cld_shdw)

def apply_cld_shdw_mask(img):
    # Subset the cloudmask band and invert it so clouds/shadow are 0, else 1.
    not_cld_shdw = img.select('cloudmask').Not()

    # Subset reflectance bands and update their masks, return the result.
    return img.select('B.*').updateMask(not_cld_shdw)

# https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR_HARMONIZED
def apply_scale_factors(image):
    optical_bands = image.select('B.*').divide(10000)
    return image.addBands(optical_bands, None, True)

In [8]:
# https://developers.google.com/earth-engine/tutorials/community/sentinel-2-s2cloudless
AOI = Emuer.geometry()    # 感兴趣区域
START_MONTH = 9           # 图像集合开始月份
END_MONTH = 11            # 图像集合结束月份，包含11月31日之前
CLOUD_FILTER = 30         # 影像集合中允许的最大影像云覆盖率百分比
CLD_PRB_THRESH = 50       # 云概率 （%）;大于 CloudWatch 的值被视为 Cloud
NIR_DRK_THRESH = 0.15     # 近红外反射率;小于该值被视为潜在云阴影
CLD_PRJ_DIST = 1          # 从云边缘搜索云阴影的最大距离（km）
BUFFER = 50               # 扩张云识别对象边缘的距离 （m）

In [9]:
s2_sr_mask = (
    get_s2_sr_cld_col(AOI, START_MONTH, END_MONTH)
    .map(add_cld_shdw_mask)
    .map(apply_cld_shdw_mask)
    .map(apply_scale_factors)
    .map(lambda image:image.select(["B2","B3","B4","B5","B6","B7","B8","B11","B12"]))
    .map(lambda image:image.clip(Emuer))
)
print("Useful Images", s2_sr_mask.size().getInfo())

Useful Images 803


In [11]:
# Map = geemap.Map()
# visualization = {
#     'min': 0.0,
#     'max': 0.3,
#     'bands': ['B4', 'B3', 'B2'],
# }
# Map.set_center(122.74791, 53.09709, 9)
# Map.add_layer(s2_sr_mask.filterBounds(Mohe.geometry().centroid()).first(), visualization, 'RGB');Map

In [12]:
import geopandas
import pandas

In [ ]:
# %conda install -y openpyxl -c conda-forge
samplingPoint = pandas.read_excel("./Samp.xlsx")

ImportError: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.

Channels:
 - conda-forge
 - defaults
Platform: win-64
Solving environment: ...working... done

## Package Plan ##

  environment location: c:\Users\Qiao\miniforge3\envs\gee

  added / updated specs:
    - openpyxl


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    et_xmlfile-2.0.0           |     pyhd8ed1ab_1          21 KB  conda-forge
    openpyxl-3.1.5             |  py313he57e174_1         472 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         494 KB

The following NEW packages will be INSTALLED:

  et_xmlfile         conda-forge/noarch::et_xmlfile-2.0.0-pyhd8ed1ab_1 
  openpyxl           conda-forge/win-64::openpyxl-3.1.5-py313he57e174_1 

The following packages will be SUPERSEDED by a higher-priority channel:

  certifi            anaconda/pkgs/main/win-64::certifi-20~ --> conda-forge/noarch::cer



==> WARNING: A newer version of conda exists. <==
    current version: 24.11.3
    latest version: 25.1.1

Please update conda by running

    $ conda update -n base -c conda-forge conda


